<a href="https://colab.research.google.com/github/kavyajeetbora/monitoring_water_surface_area/blob/master/notebooks/Plot%20the%203D%20terrain%20from%20DEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimating the lake depth


In [1]:
!pip install -q rioxarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 30.8 MB/s eta 0:00:00


In [2]:
import geemap
import ee
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rxr
import numpy as np
import plotly.graph_objects as go
import scipy as sp

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Area of Interest

Used this tool to create a geojson file of the area of interest:

[keene Polyline Tool](https://www.keene.edu/campus/maps/tool/?coordinates=77.1200409%2C%2011.5324541%0A76.9923248%2C%2011.5062217%0A76.9916382%2C%2011.3467571%0A77.1529998%2C%2011.4261642%0A77.1200409%2C%2011.5324541) : Use this tool to create a polygon and generate a GeoJson text for further use

In [50]:
geojson  = {
  "coordinates": [
    [
      [
        91.7843056,
        26.1960062
      ],
      [
        91.7582989,
        26.1955441
      ],
      [
        91.7451668,
        26.1849926
      ],
      [
        91.7316055,
        26.1772131
      ],
      [
        91.730175,
        26.1650935
      ],
      [
        91.7433071,
        26.1432647
      ],
      [
        91.7656231,
        26.1314755
      ],
      [
        91.7870808,
        26.1312444
      ],
      [
        91.788969,
        26.1308591
      ],
      [
        91.8041611,
        26.1422631
      ],
      [
        91.8041611,
        26.1422631
      ],
      [
        91.8176365,
        26.1543594
      ],
      [
        91.8238163,
        26.1748509
      ],
      [
        91.8238163,
        26.1748509
      ],
      [
        91.7997837,
        26.1835548
      ],
      [
        91.7843056,
        26.1960062
      ]
    ]
  ],
  "type": "Polygon"
}

geometry = ee.Geometry(geojson)

## Downloading the Global lakes bathymetry dataset

[Global lakes bathymetry dataset](https://gee-community-catalog.org/projects/globathy/): bathymetric data of 1.4+ million waterbodies to align with the well-established global dataset, HydroLAKES. GLOBathy uses a GIS-based framework to generate bathymetric maps based on the waterbody maximum depth estimates and HydroLAKES geometric/geophysical attributes of the waterbodies. The maximum depth estimates are validated at 1,503 waterbodies, making use of several observed data sources

In [52]:
globathy = ee.Image("projects/sat-io/open-datasets/GLOBathy/GLOBathy_bathymetry")
globathy = globathy.multiply(-1).rename('DSM').unmask(0) ## Multiplying -1 to represent the data in negative

Map = geemap.Map(basemap='OpenTopoMap')
visParams = {"min": -20, "max": 0, 'palette': ['#d73027','#f46d43','#fdae61','#fee090','#e0f3f8','#abd9e9','#74add1','#4575b4']}
Map.addLayer(globathy.clip(geometry), visParams, 'Global Bathymetry')
Map.centerObject(geometry, zoom=12)
Map

Map(center=[26.163728983392012, 91.77528600488453], controls=(WidgetControl(options=['position', 'transparent_…

## Load the elevation model

[All you need to know about digital elevation models](https://up42.com/blog/everything-you-need-to-know-about-digital-elevation-models-dem-digital)

- Load the elevation model
- add the lake bathymetry depths to the elevation model to get the lake depressions

In [53]:
alos_dsm = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2")\
.filter(ee.Filter.bounds(geometry))

alos_image = alos_dsm.first().select("DSM")

## Add the two images
terrain = alos_image.add(globathy)

In [54]:
Map = geemap.Map(basemap='OpenTopoMap')
palette = ['#543005','#8c510a','#bf812d','#dfc27d','#f6e8c3','#c7eae5','#80cdc1','#35978f','#01665e','#003c30']
visParams = {"bands": ['DSM'], "min": 0, "max": 500, 'palette': palette}

Map.addLayer(terrain.clip(geometry), visParams, 'ALOS Global DSM-30m')
# Map.addLayer(geometry, {}, "boundary")
Map.centerObject(geometry, zoom=12)
Map

Map(center=[26.163728983392012, 91.77528600488453], controls=(WidgetControl(options=['position', 'transparent_…

In [55]:
gaussianFilter = ee.Kernel.gaussian(radius=50, sigma=10, units='pixels', normalize=True)
# Apply Gaussian smoothing to the elevation data
smoothed_elevation = terrain.convolve(gaussianFilter)

In [56]:
Map = geemap.Map(basemap='OpenTopoMap')
palette = ['#543005','#8c510a','#bf812d','#dfc27d','#f6e8c3','#c7eae5','#80cdc1','#35978f','#01665e','#003c30']
visParams = {"bands": ['DSM'], "min": 0, "max": 500, 'palette': palette}

Map.addLayer(smoothed_elevation.clip(geometry), visParams, 'ALOS Global DSM-30m')
Map.centerObject(geometry, zoom=12)
Map

Map(center=[26.163728983392012, 91.77528600488453], controls=(WidgetControl(options=['position', 'transparent_…

## Export the `ee.Image` to tif file

[Projections in earth engine](https://developers.google.com/earth-engine/guides/projections)


There might be some CRS code that might not be recongnizable by google earth engine. In this case we need to define the projection using the `ee.Projection` function.

To import a new crs system you can go to epgs.io website and grab the OGC Well Known Text Definitiion of the projection system and copy paste it here:

In [57]:
projFromWKT = ee.Projection(
    '''PROJCS["WGS 84 / UTM zone 46N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",93],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32646"]]'''
)
projFromWKT

In [58]:
geemap.ee_export_image(
    terrain.reproject(projFromWKT), filename="terrain.tif", scale=30, region=geometry, file_per_band=False
)

Generating URL ...
Please wait ...
An error occurred while downloading.


## Visualize the terrain in 3D

- Convert the tif image to xarray.DataArray
- Smoothen the terrain based on user input using gaussian filter
- Plot the 3D terrain using plotly

In [59]:
def smoothen_dataArray(tif_file, sigma = 10):
    da = rxr.open_rasterio(filename=tif_file)
    da_vals = da.isel(band=0).values ## Select the first band of the image
    # Apply gaussian filter, with sigmas as variables. Higher sigma = more smoothing and more calculations. Downside: min and max values do change due to smoothing
    sigma = [sigma, sigma] ## Sigma values in x and y direction
    z_smoothed = sp.ndimage.gaussian_filter(da_vals, sigma)

    da.data = np.expand_dims(z_smoothed, axis=0)
    return da

def plot_3D_terrain(xr_array):

    Z = xr_array.values
    Y = xr_array['y'].values
    X = xr_array['x'].values

    fig = go.Figure()

    fig.add_trace(
        go.Surface(
            z=Z,
            x=X,
            y=Y,
            colorscale = 'Viridis'
        )
    )

    # Set the box aspect ratio (equal scales for all axes)
    fig.update_scenes(aspectratio=dict(x=5, y=5,z=1))
    # Remove all margins

    camera = dict(
        up=dict(x=0, y=0, z=1),
        eye=dict(x=1.25, y=1.25, z=1.25)
    )

    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        scene_camera = camera
    )

    fig.update_xaxes(showticklabels=False, title='')
    fig.update_yaxes(showticklabels=False, title='')

    return fig

In [60]:
ds = smoothen_dataArray('terrain.tif', sigma=5)
da = ds.isel(band=0)
fig = plot_3D_terrain(da)
fig.show()

References:
1. Khazaei, Bahram; Read, Laura K; Casali, Matthew; Sampson, Kevin M; Yates, David N (2022): GLOBathy Bathymetry Rasters. figshare.
Dataset. https://doi.org/10.6084/m9.figshare.13404635.v1

2. [Plotting xarrays](https://docs.xarray.dev/en/latest/user-guide/plotting.html)

3. [Improving the 3D terrain visualization from GeoTIFF using python and matplotlib](https://stackoverflow.com/q/74995160/6819442)


Fix this issue:

Using `geemap.ee_to_xarray`:

[Converting Earth Engine dataset into XArray #1942](https://github.com/gee-community/geemap/discussions/1942)

ee.Image is getting converted to the xrray but the values are converted as null values.